In [1]:
!pip install praw

  Using cached praw-7.7.0-py3-none-any.whl (189 kB)
  Using cached prawcore-2.3.0-py3-none-any.whl (16 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)
  Using cached websocket_client-1.5.1-py3-none-any.whl (55 kB)
  Using cached requests-2.29.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)


In [8]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 13.1 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 3.5 MB/s eta 0:00:0000:0100:01


In [2]:
import praw

In [3]:
reddit = praw.Reddit(client_id = "JgHZRyaH_Ya0sSPJ4WB4ZA", client_secret="yNybsXSQCZVR-r-2mo0Ribuae-zKgw", user_agent="scrape")

In [4]:
subreddit_name = "IAmA"

In [5]:
subreddit = reddit.subreddit(subreddit_name)

In [6]:
# Get top 1000 posts from subreddit
top_subreddit = subreddit.top(limit=1000)

# Create a list of posts
posts = []
for post in top_subreddit:
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])

In [9]:
# Create a dataframe from the list of posts
import pandas as pd
import numpy as np

posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [10]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,"I am Barack Obama, President of the United Sta...",216186,z1c9z,IAmA,https://www.reddit.com/r/IAmA/comments/z1c9z/i...,22903,"Hi, I’m Barack Obama, President of the United ...",1.346270e+09
1,Protect Net Neutrality. Save the Internet.,201657,7eojwf,IAmA,https://www.battleforthenet.com/,2713,,1.511329e+09
2,"[AMA Request] Victoria, ex-AMA mod",126772,3c0iw2,IAmA,https://www.reddit.com/r/IAmA/comments/3c0iw2/...,2786,**My 6 Questions:**\n\n1. How did you enjoy yo...,1.435945e+09
3,FCC just voted to kill Net Neutrality. Now we ...,121801,7jtes4,IAmA,https://www.reddit.com/r/IAmA/comments/7jtes4/...,2609,The FCC just voted to throw out the Net Neutra...,1.513275e+09
4,"I’m American citizen, undecided voter, loving ...",116911,57dw9a,IAmA,https://www.reddit.com/r/IAmA/comments/57dw9a/...,16684,"Hello Reddit,\n\nI’m just a normal guy, who sp...",1.476408e+09
...,...,...,...,...,...,...,...,...
993,[AMA Request] The Duke University scientists w...,11716,6xtwmm,IAmA,https://www.reddit.com/r/IAmA/comments/6xtwmm/...,152,This is an amazing discovery! Thank you for yo...,1.504454e+09
994,[AMA Request] Chris Pratt,11684,2cok8r,IAmA,https://www.reddit.com/r/IAmA/comments/2cok8r/...,597,The amount of Chris Pratt I've seen on reddit ...,1.407241e+09
995,IamA 90 year old coal miner and WW2 and Korean...,11685,3ugwhx,IAmA,https://www.reddit.com/r/IAmA/comments/3ugwhx/...,1787,I'm his grandson and will be fielding the ques...,1.448635e+09
996,"We are engineers from Zipline, the largest aut...",11316,120tytk,IAmA,https://www.reddit.com/r/IAmA/comments/120tytk...,804,**EDIT: Thanks everyone for your questions! We...,1.679684e+09


In [13]:
# Save dataframe to csv
posts.to_csv('1000_ama_posts.csv', index=False)

OSError: [Errno 30] Read-only file system: '1000_ama_posts.csv'